In [46]:
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient

endpoint = "https://question-answering-test-simplon.cognitiveservices.azure.com/"
credential = AzureKeyCredential("e518b6bec9204e41b9cf4b968ddc83f6")
knowledge_base_project = "Projet-X"
deployment = "production"

def main():
    client = QuestionAnsweringClient(endpoint, credential)
    with client:
        question="How much battery life do I have left?"
        output = client.get_answers(
            question = question,
            project_name=knowledge_base_project,
            deployment_name=deployment
        )
    print("Q: {}".format(question))
    print("A: {}".format(output.answers[0].answer))

if __name__ == '__main__':
    main()

Q: How much battery life do I have left?
A: No answer found


In [3]:
f = open('/home/apprenant/Bureau/NLP_QA_Project/Dataset/43_Arya_Stark.txt')
text =f.read()[:5000]

In [9]:
import os
from azure.core.credentials import AzureKeyCredential
from azure.ai.language.questionanswering import QuestionAnsweringClient
from azure.ai.language.questionanswering import models as qna
from dotenv import load_dotenv

load_dotenv()
endpoint = "https://question-answering-test-simplon.cognitiveservices.azure.com/"
credential = AzureKeyCredential(os.getenv('credential'))
def get_answer(text,question):
    client = QuestionAnsweringClient(endpoint, credential)
    with client:
        input = qna.AnswersFromTextOptions(
            question=question,
            text_documents=[text]
        )


        output = client.get_answers_from_text(input)

    best_answer = [a for a in output.answers if a.confidence > 0.5][0]
    print(u"Q: {}".format(input.question))
    print(u"A: {}".format(best_answer.answer))
    print("Confidence Score: {}".format(output.answers[0].confidence))

get_answer(bladjakjdazkljdklazjdklazjdlazjdkl love is life'', ' What is love ?')


Q:  What is love ?
A: bladjakjdazkljdklazjdklazjdlazjdkl,
           love is life
Confidence Score: 0.21424047648906708


In [25]:
import PyPDF2
 
#create file object variable
#opening method will be rb
file=open('/home/apprenant/Bureau/NLP_QA_Project/Dataset/lab_eval.pdf','rb')
 
#create reader variable that will read the pdffileobj
reader = PyPDF2.PdfReader(file)
liste = []
#This will store the number of pages of this pdf file
x= len(reader.pages)
print(x)
for i in range(x) :
    page = reader.pages[i]
    text = page.extract_text()
    liste.append(text)

file1=open(r"/home/apprenant/Bureau/NLP_QA_Project/Dataset/1.txt","a")
file1.writelines(liste)

5


12

: 

: 